In [1]:
import argparse
import datetime
import json
import random
import time
from pathlib import Path
import os, sys
import numpy as np
from utils import losses
import torch
from utils.utils import *
from scipy.special import softmax
import matplotlib.pyplot as plt
import torch.nn.functional as F

/home/sas20048/anaconda/envs/at_det/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# PSPNet_VOC, UNet_COCO10k, UNet_VOC
base_path = "./pro_data/UNet_VOC/train/"
files = os.listdir(base_path + "output")
cross_entropy_loss_func = losses.CrossEntropyLoss2d(reduction="none")
file_count = len(files)

In [3]:
check_folder_exist(base_path + "loss")
for file in range(file_count):
    output = np_read(base_path + f"output/{file}.npy")
    target = np_read(base_path + f"target/{file}.npy")
    losses = cross_entropy_loss_func(torch.from_numpy(output), torch.from_numpy(target))
    losses = losses.numpy()
    np_write(losses, base_path + f"loss/{file}.npy")
    if (file+1)%100 == 0:
        print(file)

99


In [7]:
base_path = "./pro_data/UNet_COCO/train/"
for file in range(400, 900):
    os.remove(base_path + f"output/{file}.npy")

## Generate ground truth image-based loss

In [9]:
base_path = "./pro_data/UNet_VOC/train/"
files = os.listdir(base_path + "loss")
file_count = len(files)
true_losses = None
for file in range(file_count):
    losses = np_read(base_path + f"loss/{file}.npy")
    losses = np.reshape(losses, (losses.shape[0], -1))
    losses = np.mean(losses, axis=1)
    if true_losses is None:
        true_losses = losses
    else:
        true_losses = np.concatenate((true_losses, losses), axis=0)
    if (file+1)%100 == 0:
        print(file)
np_write(true_losses, base_path + "image_true_losses.npy")

99


## Generate ground truth region-baed loss

In [10]:
## region 8x8
import torch
# base_path = "./pro_data/UNet_VOC/val/"
files = os.listdir(base_path + "output")
file_count = len(files)
avgpool = torch.nn.AdaptiveAvgPool2d((60,60))
region_based_losses = None
for file in range(file_count):
    losses = np_read(base_path + f"loss/{file}.npy")
    losses = torch.from_numpy(losses)
    losses = avgpool(losses)
    losses = losses.numpy()
    losses = losses.reshape(-1)
    if region_based_losses is None:
        region_based_losses = losses
    else:
        region_based_losses = np.concatenate((region_based_losses, losses), axis=0)
    if (file+1)%100 == 0:
        print(file)
np_write(region_based_losses, base_path + "region_8_8_true_losses.npy")

99


In [11]:
## region 16x16
import torch
# base_path = "./pro_data/UNet_VOC/val/"
files = os.listdir(base_path + "output")
file_count = len(files)
avgpool = torch.nn.AdaptiveAvgPool2d((30,30))
region_based_losses = None
for file in range(file_count):
    losses = np_read(base_path + f"loss/{file}.npy")
    losses = torch.from_numpy(losses)
    losses = avgpool(losses)
    losses = losses.numpy()
    losses = losses.reshape(-1)
    if region_based_losses is None:
        region_based_losses = losses
    else:
        region_based_losses = np.concatenate((region_based_losses, losses), axis=0)
    if (file+1)%100 == 0:
        print(file)
np_write(region_based_losses, base_path + "region_16_16_true_losses.npy")

99


In [3]:
# region 60x60
import torch
base_path = "./pro_data/UNet_VOC/train/"
files = os.listdir(base_path + "output")
file_count = len(files)
avgpool = torch.nn.AdaptiveAvgPool2d((8,8))
region_based_losses = None
for file in range(file_count):
    losses = np_read(base_path + f"loss/{file}.npy")
    losses = torch.from_numpy(losses)
    losses = avgpool(losses)
    losses = losses.numpy()
    losses = losses.reshape(-1)
    if region_based_losses is None:
        region_based_losses = losses
    else:
        region_based_losses = np.concatenate((region_based_losses, losses), axis=0)
    if (file+1)%100 == 0:
        print(file)
np_write(region_based_losses, base_path + "region_60_60_true_losses.npy")

99


In [5]:
# region 30x30
import torch
base_path = "./pro_data/UNet_VOC/val/"
files = os.listdir(base_path + "output")
file_count = len(files)
avgpool = torch.nn.AdaptiveAvgPool2d((16,16))
region_based_losses = None
for file in range(file_count):
    losses = np_read(base_path + f"loss/{file}.npy")
    losses = torch.from_numpy(losses)
    losses = avgpool(losses)
    losses = losses.numpy()
    losses = losses.reshape(-1)
    if region_based_losses is None:
        region_based_losses = losses
    else:
        region_based_losses = np.concatenate((region_based_losses, losses), axis=0)
    if (file+1)%100 == 0:
        print(file)
np_write(region_based_losses, base_path + "region_30_30_true_losses.npy")

99


In [7]:
# region 32x32
import torch
base_path = "./pro_data/UNet_VOC/train/"
files = os.listdir(base_path + "output")
file_count = len(files)
avgpool = torch.nn.AdaptiveAvgPool2d((15,15))
region_based_losses = None
for file in range(file_count):
    losses = np_read(base_path + f"loss/{file}.npy")
    losses = torch.from_numpy(losses)
    losses = avgpool(losses)
    losses = losses.numpy()
    losses = losses.reshape(-1)
    if region_based_losses is None:
        region_based_losses = losses
    else:
        region_based_losses = np.concatenate((region_based_losses, losses), axis=0)
    if (file+1)%100 == 0:
        print(file)
np_write(region_based_losses, base_path + "region_32_32_true_losses.npy")

99


In [8]:
region_based_losses.max(), region_based_losses.min()

(15.501279, 0.0)

## Entropy part

In [12]:
# PSPNet_VOC, UNet_COCO10k, UNet_VOC
# base_path = "./pro_data/UNet_VOC/val/"
files = os.listdir(base_path + "output")
file_count = len(files)
check_folder_exist(base_path + "entropy")
for file in range(file_count):
    output = np_read(base_path + f"output/{file}.npy")
    output = torch.from_numpy(output)
    output = F.softmax(output, dim=1)
    entropy = torch.sum(torch.mul(-output, torch.log(output + 1e-20)), dim=1).unsqueeze(dim=1)
    np_write(entropy.numpy(), base_path + f"entropy/{file}.npy")
    if (file+1)%100 == 0:
        print(file)

99


## Feature Part

In [13]:
# PSPNet_VOC, UNet_COCO10k, UNet_VOC
# base_path = "./pro_data/UNet_VOC/val/"
files = os.listdir(base_path + "output")
file_count = len(files)
check_folder_exist(base_path + "feature")
for file in range(file_count):
    output = np_read(base_path + f"output/{file}.npy")
    output = torch.from_numpy(output)
    output = F.softmax(output, dim=1)
    np_write(output.numpy(), base_path + f"feature/{file}.npy")
    if (file+1)%100 == 0:
        print(file)

99


In [4]:
## split feature
base_path = "./pro_data/UNet_COCO10k/train/"
check_folder_exist(base_path + "feature_split")
files = os.listdir(base_path + "feature")
file_count = len(files)
count = 0
for file in range(file_count):
    feature = np_read(base_path + f"feature/{file}.npy")
    samples = feature.shape[0]
    for i in range(samples):
        np_write(feature[i], base_path + f"feature_split/{count}.npy")
        count += 1
    if (file+1)%100 == 0:
        print(file)

99
199
299
399
499
599
699
799
899
999
1099


## Check loss range

In [5]:
base_path = "./pro_data/PSPNet_VOC_pre/val/"
image_true_losses = np_read(base_path + "image_true_losses.npy")
print(image_true_losses.min(), image_true_losses.max())
region_16_16_true_losses = np_read(base_path + "region_16_16_true_losses.npy")
print(region_16_16_true_losses.min(), region_16_16_true_losses.max())
region_8_8_true_losses = np_read(base_path + "region_8_8_true_losses.npy")
print(region_8_8_true_losses.min(), region_8_8_true_losses.max())
# region_8_8_true_losses = np_read(base_path + "region_32_32_true_losses.npy")
# print(region_8_8_true_losses.min(), region_8_8_true_losses.max())

0.00037822526 0.466109
0.0 8.249905
0.0 10.661509


FileNotFoundError: [Errno 2] No such file or directory: './pro_data/PSPNet_VOC_pre/val/region_32_32_true_losses.npy'

In [6]:
base_path = "./pro_data/PSPNet_VOC_pre/train/"
image_true_losses = np_read(base_path + "image_true_losses.npy")
print(image_true_losses.min(), image_true_losses.max())
region_16_16_true_losses = np_read(base_path + "region_16_16_true_losses.npy")
print(region_16_16_true_losses.min(), region_16_16_true_losses.max())
region_8_8_true_losses = np_read(base_path + "region_8_8_true_losses.npy")
print(region_8_8_true_losses.min(), region_8_8_true_losses.max())
# region_8_8_true_losses = np_read(base_path + "region_32_32_true_losses.npy")
# print(region_8_8_true_losses.min(), region_8_8_true_losses.max())

0.00017570611 0.5550945
0.0 8.893623
0.0 9.565056


## Generate 4 splited loss ground truth

In [37]:
base_path = "./pro_data/PSPNet_VOC/val/"
files = os.listdir(base_path + "output")

In [38]:
import torch
avgpool = torch.nn.AdaptiveAvgPool2d((2,2))
region_based_losses = None
for file in files:
    losses = np_read(base_path + f"loss/{file}")
    losses = torch.from_numpy(losses)
    losses = avgpool(losses)
    losses = losses.numpy()
    losses = losses.reshape(-1)
    if region_based_losses is None:
        region_based_losses = losses
    else:
        region_based_losses = np.concatenate((region_based_losses, losses), axis=0)

In [39]:
np_write(region_based_losses, base_path + "image_split_2_2_true_losses.npy")

In [41]:
region_based_losses.max(), region_based_losses.min()

(1.0793692, 0.0)

## Generate 8x8 patch ground truth for 4 splited image

In [46]:
base_path = "./pro_data/PSPNet_VOC/val/"
files = os.listdir(base_path + "output")

In [47]:
import torch
avgpool = torch.nn.AdaptiveAvgPool2d((60,60))
splited_location_list = [[0,30,0,30], [0,30,30,60],[30,60,0,30],[30,60,30,60]]
region_based_losses = None
for file in files:
    losses = np_read(base_path + f"loss/{file}")
    losses = torch.from_numpy(losses)
    losses = avgpool(losses)
    losses = losses.numpy()
    for i in range(losses.shape[0]):
        for j in range(4):
            location = splited_location_list[j]
            loss = losses[i, location[0]:location[1],location[2]:location[3]]
            loss = loss.reshape(-1)
            if region_based_losses is None:
                region_based_losses = loss
            else:
                region_based_losses = np.concatenate((region_based_losses, loss), axis=0)

In [48]:
np_write(region_based_losses, base_path + "region_8_8_image_split_true_losses.npy")

In [50]:
region_based_losses.shape, region_based_losses.max(), region_based_losses.min()

((5216400,), 10.661509, 0.0)